In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/france-road-merged/merged-vehicles-users-places-characteristics_2025_01_06_2138.csv
/kaggle/input/france-road-merged/preprocessedMerged-_2025_01_20_1655.parquet
/kaggle/input/france-road-merged/merged-vehicles-users-places-characteristics_2025_01_06_2141.parquet
/kaggle/input/france-road-merged/merged-vehicles-users-places-characteristics_2025_01_06_2140.feather


In [2]:
%%time
## all imports
import polars as pl
import pyarrow.parquet as pq
import dask.dataframe as dd
import os
import shutil
import json
from enum import Enum
from datetime import datetime
from ydata_profiling import ProfileReport
from pathlib import Path
import random
import hdbscan
import pyarrow

from dask.distributed import LocalCluster
##Metrics
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.metrics import mutual_info_score, adjusted_rand_score
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

import plotly.express as px
#For excel stuff
import openpyxl
from openpyxl.drawing.image import Image


# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Concurrency
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
import time

#  Warnings
import warnings
warnings.filterwarnings('ignore')

# Set random state
random_state = 42
# Set figure size
plt.rcParams["figure.figsize"] = (20, 20)


class ExtensionMethods:
    @staticmethod
    def generate_filename(filename=None,extension=None):
        current_datetime = datetime.now()
        f = current_datetime.strftime("%Y_%m_%d_%H%M")
        if (filename is None) or (extension is None):
            return str(f)
        else:
            stitched_f = str(filename)+"_"+str(f)+"."+str(extension)
            return str(stitched_f)

    @staticmethod
    def get_file_name_without_extension(filename):
        if filename == None:
            return "Provide a file"
        return Path(filename).stem


feather_file = '/kaggle/input/france-road-merged/merged-vehicles-users-places-characteristics_2025_01_06_2140.feather'
parquet_file = '/kaggle/input/france-road-merged/merged-vehicles-users-places-characteristics_2025_01_06_2141.parquet'
csv_file = '/kaggle/input/france-road-merged/merged-vehicles-users-places-characteristics_2025_01_06_2138.csv'
pre_file= '/kaggle/input/france-road-merged/preprocessedMerged-_2025_01_20_1655.parquet'


def explore_dataframe(df):
    """
    Prints basic information about the dataframe.
    Args:
        df : Data frame
        #TODO : exception handling later
    """

    # Basic information
    print("DataFrame Info:")
    df.info()

    #Null counts
    print("\nNull Values per Column:")
    null_counts = df.isnull().sum()
    print(null_counts)

    # Normal values per column
    print("\nNormal Values per Column:")
    normal_counts = df.notnull().sum()
    print(normal_counts)

    # Descriptive stuff
    print("\nDescriptive Statistics:")
    print(df.describe(include='all'))

    # Value counts for cat stuff
    categorical_cols = df.select_dtypes(include=['object']).columns
    print("\nValue Counts for Categorical Columns:")
    for col in categorical_cols:
        print(f"\nColumn: {col}")
        print(df[col].value_counts())

ModuleNotFoundError: No module named 'hdbscan'

In [3]:
df = pd.read_parquet(pre_file)
df.isna().sum()

NameError: name 'pre_file' is not defined

In [ ]:
class CategoryBaseEnum(Enum):
    @classmethod
    def Name(cls):
        return f"{cls.__name__}"
    @classmethod
    def IsCategory(cls):
        return(True if len(cls.__members__)>0 else False)
    @classmethod
    def get_description(cls):
        return cls.__doc__ or "No description available"
    @classmethod
    def to_dict(cls):
        return {member.name:member.value for member in cls}
    @classmethod
    def to_json(cls, indent=4):
        return json.dumps(cls.to_dict(), indent=indent)

In [ ]:
class FileTypeEnum(Enum):
    @classmethod
    def Name(cls):
        return f"{cls.__name__}"
    @classmethod
    def get_description(cls):
        return cls.__doc__ or "No description available"
    @classmethod
    def to_dict(cls):
        return {
            member.name:member.value.to_dict() for member in cls}

In [ ]:
%%time
class Road_Accident_Enum(FileTypeEnum):
    '''Collection of all the categories for file type 'vehicle.csv' '''

    class Num_Acc(CategoryBaseEnum):
        '''The Index/Number of the Crash follows the pattern yyyyxxxxx and is the index column''' 
    class id_vehicule(CategoryBaseEnum):
        '''The vehicle id  in terms of xxx-xxx'''
    
    class catv(CategoryBaseEnum): # catv Vehicle Category
        '''The Category of Vehicle involved in the crash'''
        UNDETERMINED = 0
        BICYCLE = 1
        MOPED_LESS_EQUAL_50CC = 2
        MICROCAR = 3
        TOURISM_VEHICLE = 7
        UTILITY_VEHICLE_PTAC_1_5T_3_5T = 10 #1.5<PTAC<3.5
        HEAVY_TRUCK_PTAC_3_5T_7_5T = 13
        HEAVY_TRUCK_PTAC_OVER_7_5T = 14
        HEAVY_TRUCK_OVER_3_5T_WITH_TRAILER = 15
        TRACTOR_ONLY = 16
        TRACTOR_WITH_SEMI_TRAILER = 17
        SPECIAL_VEHICLE = 20
        AGRICULTURAL_TRACTOR = 21
        SCOOTER_LESS_EQUAL_50CC = 30
        MOTORCYCLE_50CC_125CC = 31
        SCOOTER_50CC_125CC = 32
        MOTORCYCLE_OVER_125CC = 33
        SCOOTER_OVER_125CC = 34
        LIGHT_QUAD_LESS_EQUAL_50CC = 35
        HEAVY_QUAD_OVER_50CC = 36
        BUS = 37
        COACH = 38
        TRAIN = 39
        TRAMWAY = 40
        THREE_WHEELED_VEHICLE_LESS_EQUAL_50CC = 41
        THREE_WHEELED_VEHICLE_50CC_125CC = 42
        THREE_WHEELED_VEHICLE_OVER_125CC = 43
        MOTORIZED_PERSONAL_TRANSPORT = 50
        NON_MOTORIZED_PERSONAL_TRANSPORT = 60
        OTHER_VEHICLE = 99

    class obs(CategoryBaseEnum):  #obs Static ObstacleHit
        ''' The Static/Stationary Obstacle Hit'''
        UNKNOWN = 0
        PARKED_VEHICLE = 1
        TREE_ON_ROADSIDE = 2
        METAL_BARRIER = 3
        CONCRETE_BARRIER = 4
        OTHER_BARRIER=5
        BUILDING_WALL_BRIDGE_PIER = 6 #BUILDING, WALL OR BRIDGE PIER
        VERTICAL_SIGNPOST_OR_EMERGENCY_CALLBOX = 7
        POLE = 8
        URBAN_FURNITURE = 9 # NO CLUE WHAT URBAN FURNITURE
        PARAPET = 10
        REFUGE_ISLAND_BOLLARD = 11 #THE ROAD ISLAND / BOLLARD
        SIDEWALK = 12 #SIDEWALK OR CURB
        DITCH = 13 #DITCH OR EMBANKMENT
        OTHER_OBS_ON_ROADWAY = 14
        OTHER_OBS_ON_SIDEWALK = 15
        ROADWAY_EXIT_WITOUT_OBSTACLES = 16
        AQUEDUCT_HEAD = 17

    class obsm(CategoryBaseEnum): #obsm #Mobile obstacle hit
        '''The Dynamic Obstacle Hit'''
        UNKNOWN = 0
        PEDESTRIAN =1 
        VEHICLE = 2
        RAIL_VEHICLE = 3
        ANIMAL_DOMESTIC = 4
        ANIMAL_WILD = 5
        OTHER = 9
        
    class choc(CategoryBaseEnum): #choc Initial Point of Impact
        '''The Initial Point of Impact of the crash'''
        UNKNOWN = 0
        FRONT = 1
        FRONT_LEFT = 2
        FRONT_RIGHT = 3
        REAR = 4
        REAR_RIGHT = 5
        REAR_LEFT = 6
        SIDE_LEFT = 7
        SIDE_RIGHT = 8
        MULTIPLE = 9

    
    class manv(CategoryBaseEnum): #manv , Main action before crash
        '''The Main Action performed by the user before the crash'''
        UNKNOWN = 0
        CIRCULATING_NO_DIRECTION_CHANGE =1
        CIRCULATING_SAME_DIRECTION = 2 #SAME DIRECTION SAME LANE
        CIRCULATION_BETWEEN_2_LANES = 3
        CIRCULATING_REVERSING = 4
        CIRCULATING_AGAINTS_FLOW_TRAFFIC = 5
        CIRCULATING_CROSSING_MEDIAN_STRIP = 6
        CIRCULATING_IN_BUSLANE_SAME_DIRECTION = 7
        CIRCULATING_IN_BUSLANE_OPP_DIRECTION = 8
        CIRCULATING_INSERTION =9
        CIRCULATING_TURNING_AROUND_CARRIAGE_WAY = 10
        CHANGING_LANE_LEFT = 11
        CHANGING_LANE_RIGHT = 12
        DEPORT_LEFT = 13
        DEPORT_RIGHT = 14
        TURNING_LEFT = 15
        TURNING_RIGHT = 16
        OVERTAKING_LEFT = 17
        OVERTAKING_RIGHT = 18 
        #VARIOUS S
        CROSSING_ROAD = 19
        PARKING_ACTION = 20
        AVOIDANCE_ACTION = 21
        DOOR_OPENED = 22
        STOP_NO_PARKING = 23
        PARKED_WITH_PASS = 24 # PARKED WITH PASSANGERS
        DRIVING_SIDEWALK = 25
        OTHER_ACTIONS = 26
        
    
    class motor(CategoryBaseEnum): # motor
        '''The Type of Motor(In terms of fuel type) involved in the crash'''
        UNKNOWN = 0
        CONVENTIONAL_FUEL = 1 # PETROL, DIESEL ,ETC
        HYBRID_ELECTRIC = 2
        ELECTRIC = 3
        HYDROGEN = 4
        HUMAN_POWERED = 5
        OTHER = 6

    class catr(CategoryBaseEnum):
        '''Category of the road'''
        MOTORWAY = 1
        NATIONAL_ROAD = 2
        DEPARTMENTAL_ROAD = 3
        MUNICIPAL_ROAD = 4
        OFF_NETWORK = 5
        PARKING_AREA = 6
        URBAN_METROPOLE_ROAD = 7
        OTHER = 9

    class circ(CategoryBaseEnum):
        '''Traffic regime:'''
        NOT_PROVIDED = -1
        ONE_WAY = 1
        BIDIRECTIONAL = 2
        SEPARATE_LANES = 3
        VARIABLE_LANE = 4
       
    class vosp(CategoryBaseEnum):
        '''Indicates the existence of a reserved lane, regardless of whether or not the accident occurs on this lane.'''
        NOT_PROVIDED = -1
        NOT_APPLICABLE = 0
        CYCLE_TRACK = 1
        CYCLE_LANE = 2
        RESERVED_LANE = 3
        
    class prof(CategoryBaseEnum):
        '''Longitudinal profile describes the slope of the road at the location of the accident:'''
        NOT_PROVIDED = -1
        FLAT = 1
        SLOPE = 2
        CREST = 3
        BOTTOM = 4

    class plan(CategoryBaseEnum):
        '''Plan layout'''
        NOT_PROVIDED = -1
        STRAIGHT_PART = 1
        CURVE_LEFT = 2
        CURVE_RIGHT = 3
        S_SHAPE = 4
        
    class surf(CategoryBaseEnum):
        '''Surface condition'''
        NOT_PROVIDED = -1
        NORMAL = 1
        WET = 2
        PUDDLES = 3
        FLOODED = 4
        SNOWY = 5
        MUDDY = 6
        ICY = 7
        GREASE = 8
        OTHER = 9
       
    class infra(CategoryBaseEnum):
        '''Development - Infrastructure:'''
        NOT_PROVIDED = -1
        NONE = 0
        UNDERGROUND = 1
        BRIDGE = 2
        INTERCHANGE_RAMP = 3
        RAILROAD = 4
        AMENAGED_CROSSROAD = 5
        PEDESTRIAN_ZONE = 6
        TOLL_ZONE = 7
        WORKZONE = 8
        OTHERS = 9
        

    class situ(CategoryBaseEnum):
        '''Situation of the accident'''
        NOT_PROVIDED = -1
        NONE = 0
        ON_ROADWAY = 1
        ON_EMERGENCY_LANE = 2
        ON_SHOULDER = 3
        ON_SIDEWALK = 4
        ON_CYCLE_PATH = 5
        ON_OTHER_SPECIAL_LANE = 6
        OTHERS = 8

    class vma(CategoryBaseEnum):
        '''Maximum speed permitted at the location and time of the accident.'''

    
    class catu(CategoryBaseEnum): # 
        '''User category'''
        UNDETERMINED = 0
        DRIVER = 1
        PASSENGER = 2
        PEDESTRIAN = 3
        
       
    class grav(CategoryBaseEnum):  
        ''' Severity of the accident: The injured users are classified into three categories of victims plus the uninjured'''
        UNKNOWN = 0
        NO_INJURY = 1
        KILLED = 2
        INJURED_HOSPITALIZED = 3
        MINOR_INJURY = 4
        
       
    class sexe(CategoryBaseEnum):
        '''User's gender'''
        UNKNOWN = 0
        MALE =1 
        FEMALE = 2
        
       
    class an_nais(CategoryBaseEnum): 
        '''Year of birth of the user'''

    class trajet(CategoryBaseEnum):
        '''Reason for traveling at the time of the accident'''
        NOT_PROVIDED = 0  
        HOME_TO_WORK = 1  
        HOME_TO_SCHOOL = 2  
        SHOPPING = 3  
        PROFESSIONAL_USE = 4  
        WALK_OR_LEISURE = 5  
        OTHER = 9  
        
  
    class secu1(CategoryBaseEnum):
        '''The existence of a safety equipment'''
        NOT_PROVIDED = -1  
        NO_EQUIPMENT = 0  
        SEATBELT = 1  
        HELMET = 2  
        CHILD_DEVICE = 3  
        REFLECTIVE_VEST = 4  
        AIRBAG_2W_3W = 5  
        GLOVES_2W_3W = 6  
        GLOVES_AIRBAG_2W_3W = 7  
        NOT_DETERMINABLE = 8  
        OTHER = 9  
            

    class jour(CategoryBaseEnum):
        '''Day of the accident'''
       
    class mois(CategoryBaseEnum):
        '''Month of the accident'''
       
    class an(CategoryBaseEnum):
        '''Year of the accident'''
       
    class hrmn(CategoryBaseEnum):
        '''Hour and minute of the accident'''
       
    class lum(CategoryBaseEnum):
        '''Lighting conditions during the accident'''
        DAYLIGHT = 1
        DUSK_OR_DAWN = 2
        NIGHT_WITHOUT_PUBLIC_LIGHT = 3
        NIGHT_WITH_PUBLIC_LIGHT_OFF = 4
        NIGHT_WITH_PUBLIC_LIGHT_ON = 5
        
       
    class dep(CategoryBaseEnum):
        '''Department code (INSEE code for French departments)'''
       
    class agg(CategoryBaseEnum):
        '''Location of the accident: inside or outside the urban area'''
        OUTSIDE_URBAN_AREA = 1
        INSIDE_URBAN_AREA = 2
        
       
    class int(CategoryBaseEnum):
        '''Type of intersection where the accident occurred'''
        NO_INTERSECTION = 1
        X_INTERSECTION = 2
        T_INTERSECTION = 3
        Y_INTERSECTION = 4
        INTERSECTION_WITH_MORE_THAN_4_BRANCHES = 5
        ROUNDABOUT = 6
        SQUARE = 7
        RAILWAY_CROSSING = 8
        OTHER_INTERSECTION = 9
        
       
    class atm(CategoryBaseEnum):
        '''Atmospheric conditions during the accident'''
        NOT_PROVIDED = -1
        NORMAL = 1
        LIGHT_RAIN = 2
        HEAVY_RAIN = 3
        SNOW_HAIL = 4
        FOG_SMOKE = 5
        STRONG_WIND_STORM = 6
        BLINDING_WEATHER = 7
        CLOUDY_WEATHER = 8
        OTHER = 9
        
       
    class col(CategoryBaseEnum):
        '''Type of collision'''
        NOT_PROVIDED = -1
        TWO_VEHICLES_FRONT = 1
        TWO_VEHICLES_REAR = 2
        TWO_VEHICLES_SIDE = 3
        THREE_VEHICLES_CHAIN = 4
        THREE_VEHICLES_MULTIPLE_COLLISIONS = 5
        OTHER_COLLISION = 6
        NO_COLLISION = 7
        
       
    class adr(CategoryBaseEnum):
        '''Postal address of the accident (provided for accidents inside urban areas)'''
       
    class lat(CategoryBaseEnum):
        '''Latitude of the accident location'''
       
    class long(CategoryBaseEnum):
        '''Longitude of the accident location'''


In [ ]:
valid_columns_dict = Road_Accident_Enum.to_dict()
valid_columns = valid_columns_dict.keys()
print(valid_columns)

In [ ]:
df = pd.read_csv(csv_file, decimal=',')
data = df#df.sample(frac=0.1, random_state=random_state)

In [ ]:
data = data[[col for col in data.columns if col in valid_columns]]
print(data.isna().sum())


data['hrmn'] = data['hrmn'].astype('object')
print(data.info())


In [ ]:
data['hrmn'] = data['hrmn'].astype(str).str.replace(r'\D', '', regex=True)


data['hrmn'] = data['hrmn'].str.zfill(4)  


data['hour'] = data['hrmn'].str[:2].astype(int)
data['minute'] = data['hrmn'].str[2:].astype(int)


print(data.head())

In [ ]:
missing_values = df[["an", "mois", "jour","hrmn"]].isna().sum()
print("Missing values:\n", missing_values)

df_cleaned = df.dropna(subset=["an", "mois", "jour","hrmn"])

df_cleaned['an'] = pd.to_numeric(df_cleaned['an'], errors='coerce')
df_cleaned['mois'] = pd.to_numeric(df_cleaned['mois'], errors='coerce')
df_cleaned['jour'] = pd.to_numeric(df_cleaned['jour'], errors='coerce')

df_cleaned['hrmn'] = df_cleaned['hrmn'].astype(str).str.replace(r'\D', '', regex=True).str.zfill(4)  
df_cleaned['hour'] = df_cleaned['hrmn'].str[:2].astype(int)  
df_cleaned['minute'] = df_cleaned['hrmn'].str[2:].astype(int)  

df_cleaned["an"] = df_cleaned["an"].astype(int)
df_cleaned["mois"] = df_cleaned["mois"].astype(int)
df_cleaned["jour"] = df_cleaned["jour"].astype(int)

current_year = pd.Timestamp.now().year
df_cleaned = df_cleaned[(df_cleaned["an"] >= 1900) & (df_cleaned["an"] <= current_year)]


df_cleaned = df_cleaned[(df_cleaned["mois"] >= 1) & (df_cleaned["mois"] <= 12)]

df_cleaned = df_cleaned[(df_cleaned["jour"] >= 1) & (df_cleaned["jour"] <= 31)]
def is_valid_date(row):
    try:
        pd.Timestamp(year=row["an"], month=row["mois"], day=row["jour"])
        return True
    except ValueError:
        return False


df_cleaned = df_cleaned[df_cleaned.apply(is_valid_date, axis=1)]
df_cleaned = df_cleaned.rename(columns={'an': 'year', 'mois': 'month', 'jour': 'day'})


In [ ]:
df_cleaned["datetime"] = pd.to_datetime(df_cleaned[['year','month','day','hour','minute']], errors='coerce')

df = df_cleaned.reset_index(drop=True)
print("Updated DataFrame with datetime column:")
print(df.head())

In [ ]:
from prophet import Prophet


prophet_df = df[["datetime", "grav"]].rename(columns={"datetime": "ds", "grav": "y"})


model = Prophet(seasonality_mode='multiplicative')
model.fit(prophet_df)

future = model.make_future_dataframe(periods=365)
forecast = model.predict(future)



model.plot(forecast)
plt.title("Accident Severity Forecast")
plt.xlabel("Date")
plt.ylabel("Severity Level")
plt.show()

model.plot_components(forecast)
plt.show()


In [ ]:
forecast['year'] = forecast['ds'].dt.year
fig = px.line(forecast, x='ds', y='yhat', 
              title="Forescast",
              labels={"ds": "Time Line", "yhat": "Garv Level"},
              template="plotly_dark"  
             )


fig.show()

fig.write_html("Forecasting-2024.html")

In [ ]:
%%time
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [ ]:
data = df.sample(frac=0.02, random_state=random_state)#df.head(5000)
data['datetime'] = pd.to_datetime(data['datetime'])
data['datetime']

In [ ]:
train_data = TimeSeriesDataFrame.from_data_frame(
    data,
    id_column="Num_Acc",
    timestamp_column="datetime"
)


train_data 

In [ ]:
%pip install pmdarima

In [ ]:
import pmdarima as pm
model = pm.auto_arima(data['grav'], 
                      seasonal=True,    
                      m=1,             
                      trace=True)      


print(model.summary())


model.plot_diagnostics(figsize=(10, 8))
plt.show()


if model.seasonal_order[3] == 0:  
    print("The model is likely additive.")
else:
    print("The model is likely multiplicative.")

In [ ]:
data['grav'].value_counts()
data['grav'] = data['grav']+2
data['grav'].value_counts()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

result_add = seasonal_decompose(data['grav'], model='additive', period=12)
result_add.plot()
plt.title("Additive Decomposition")
plt.show()


result_mul = seasonal_decompose(data['grav'], model='multiplicative', period=12)
result_mul.plot()
plt.title("Multiplicative Decomposition")
plt.show()


In [ ]:
## remove anything not mainland france
LAT_MIN, LAT_MAX = 40.0, 60.0
LONG_MIN, LONG_MAX = -10.0, 10.0 ## approx

mask = (data['lat'] >= LAT_MIN) & (data['lat'] <= LAT_MAX) & (data['long'] >= LONG_MIN) & (data['long'] <= LONG_MAX)

no_data = data[~mask]
data = data[mask]


print(no_data)

In [ ]:
def create_parquet(data):
    if data is None:
        raise ValueError("Data can't be None for Parquet Creation")
    obj_cols = data.select_dtypes(include =['object']).columns
    for col in obj_cols:
        data[col]=data[col].astype(str)
    filepath = os.path.join(os.getcwd(),ExtensionMethods.generate_filename("preprocessedMerged-", "parquet"))
    data.to_parquet(filepath, engine='pyarrow',compression="zstd", compression_level=10, index=False)
    print(f"\n Finished Saving parquet to: {filepath}")


def pre_processing_step(data):
    #step 0 - Only take valid_cols
    valid_columns_dict = Road_Accident_Enum.to_dict()
    valid_columns = valid_columns_dict.keys()
    data = data[[col for col in data.columns if col in valid_columns]]
    print(data.columns)
    #step 1: convert the lat and long , remove all except mainland france
    LAT_MIN, LAT_MAX = 40.0, 60.0
    LONG_MIN, LONG_MAX = -10.0, 10.0
    mask = (data['lat'] >= LAT_MIN) & (data['lat'] <= LAT_MAX) & (data['long'] >= LONG_MIN) & (data['long'] <= LONG_MAX)
    data = data[mask]
    #step 2 : take care of 'an_nais'
    data = data.dropna(subset=['an_nais'])
    data['an_nais'] = pd.to_numeric(data['an_nais'])
    #step 3: take care of addr
    data['adr'] = data['adr'].fillna("UNKNOWN")
    create_parquet(data)
    print(f"\n Finished Preprocessing")
    return data
    

In [ ]:
# age 
data['age'] = 2025 - data['an_nais']
print(data['age'])

In [ ]:
%pip install "zenml[server]" notebook

In [ ]:
from zenml import pipeline, step

@step  # Just add this decorator
def load_data() -> dict:
    training_data = [[1, 2], [3, 4], [5, 6]]
    labels = [0, 1, 0]
    return {'features': training_data, 'labels': labels}

@step
def train_model(data: dict) -> None:
    total_features = sum(map(sum, data['features']))
    total_labels = sum(data['labels'])
    
    print(f"Trained model using {len(data['features'])} data points. "
          f"Feature sum is {total_features}, label sum is {total_labels}")

@pipeline  # This function combines steps together 
def simple_ml_pipeline():
    dataset = load_data()
    train_model(dataset)

if __name__ == "__main__":
    run = simple_ml_pipeline()  # call this to run the pipeline
   